# ANOVA in R

## Overview

Analysis of Variance (ANOVA) tests whether the means of three or more groups differ significantly. It partitions total variation in the response into variation *among* groups and variation *within* groups. A significant result tells you that at least one group mean differs — post-hoc tests identify which ones.

| Test | Use Case |
|---|---|
| One-way ANOVA | One categorical predictor with 3+ levels |
| Two-way ANOVA | Two categorical predictors; tests main effects and interaction |
| Repeated measures ANOVA | Same subjects measured across 3+ conditions (use mixed models instead for complex designs) |
| Welch's ANOVA | One-way ANOVA robust to unequal variances |

> **Why not just run multiple t-tests?** Running k(k-1)/2 pairwise t-tests inflates the type I error rate. With 4 groups that is 6 tests, giving a familywise error rate of ~26% at α = 0.05. ANOVA controls this by testing all groups simultaneously.

## Applications by Sector

| Sector | Example |
|---|---|
| **Ecology** | Do mean invertebrate densities differ across three sediment treatment groups? Does mean bird abundance differ among habitat types (grassland, cropland, CRP)? |
| **Healthcare** | Do mean recovery times differ across three treatment protocols? Are post-op pain scores different among four surgical approaches? |
| **Finance** | Do mean transaction values differ across customer segments? Are mean monthly returns different across three portfolio strategies? |
| **Insurance** | Do mean claim amounts differ across policy tiers? Are mean time-to-settlement values different across regional offices? |

---

## Assumptions Checklist

Review before running. Tests for each are run inline below.

- [ ] **Independence:** Observations are independent within and across groups
- [ ] **Normality:** Residuals (not raw data) are approximately normally distributed within each group — less critical with n > 30 per group due to CLT
- [ ] **Homogeneity of variances (homoscedasticity):** Variance is approximately equal across groups — test with Levene's test
- [ ] **No extreme outliers:** Influential outliers can distort group means and inflate within-group variance

> **If normality is violated:** Use Kruskal-Wallis test (see `06_nonparametric/rank_tests_multiple_groups.ipynb`)  
> **If equal variances are violated:** Use Welch's one-way ANOVA (`oneway.test(var.equal = FALSE)`) — demonstrated below

---

## Setup

In [ ]:
# ── Libraries ────────────────────────────────────────────────────────────────
library(tidyverse)    # data manipulation and visualization
library(ggpubr)       # publication-ready plots with stat annotations
library(car)          # Levene's test; Type III SS for two-way ANOVA
library(effectsize)   # eta-squared and omega-squared effect sizes
library(rstatix)      # tidy-friendly ANOVA and post-hoc tests
library(multcomp)     # Tukey HSD and other multiple comparisons

# ── Reproducibility ──────────────────────────────────────────────────────────
set.seed(42)

## Data

We use two built-in datasets:
- `PlantGrowth`: plant yield under three treatment conditions (control, treatment 1, treatment 2) — clean one-way ANOVA example
- `ToothGrowth`: guinea pig tooth length by supplement type (OJ vs VC) and dose (0.5, 1.0, 2.0 mg) — two-way ANOVA example with interaction

In [ ]:
# ── PlantGrowth ───────────────────────────────────────────────────────────────
head(PlantGrowth)
PlantGrowth %>%
  group_by(group) %>%
  summarise(n = n(), mean = mean(weight), sd = sd(weight), se = sd/sqrt(n))

# ── ToothGrowth ───────────────────────────────────────────────────────────────
# Convert dose to factor — it is categorical here, not continuous
ToothGrowth$dose <- factor(ToothGrowth$dose)
head(ToothGrowth)
ToothGrowth %>%
  group_by(supp, dose) %>%
  summarise(n = n(), mean = mean(len), sd = sd(len), .groups = "drop")

---

## Assumptions Testing

### Outliers

In [ ]:
# ── Boxplot: visual outlier check ─────────────────────────────────────────────
ggboxplot(PlantGrowth, x = "group", y = "weight",
          fill = "group",
          palette = c("#4a8fff", "#4fffb0", "#ffd166"),
          add = "jitter") +
  labs(title = "Plant Yield by Treatment Group",
       subtitle = "Visual check for outliers",
       y = "Dry Weight (g)", x = "Treatment") +
  theme(legend.position = "none")

# ── Identify statistical outliers (> 1.5 * IQR) ──────────────────────────────
PlantGrowth %>%
  group_by(group) %>%
  rstatix::identify_outliers(weight)
# 'is.extreme = TRUE' flags values > 3 * IQR — consider investigating these

### Normality of Residuals

Test normality on model **residuals**, not on raw data. With 3+ groups and reasonable n, the CLT provides some robustness, but it is good practice to check.

In [ ]:
# ── Fit model first to extract residuals ─────────────────────────────────────
model_plant <- aov(weight ~ group, data = PlantGrowth)

# ── Shapiro-Wilk on residuals ─────────────────────────────────────────────────
shapiro.test(residuals(model_plant))
# p > 0.05: no evidence of non-normality in residuals

# ── Q-Q plot of residuals ─────────────────────────────────────────────────────
qqnorm(residuals(model_plant), main = "Q-Q Plot of Residuals")
qqline(residuals(model_plant), col = "steelblue", lwd = 2)
# Points should fall close to the diagonal

### Homogeneity of Variances

Use **Levene's test** (robust to non-normality, preferred over Bartlett's):  
- p > 0.05 → equal variances → standard ANOVA is appropriate  
- p ≤ 0.05 → unequal variances → use Welch's ANOVA (`oneway.test(var.equal = FALSE)`)

In [ ]:
# ── Levene's test ─────────────────────────────────────────────────────────────
car::leveneTest(weight ~ group, data = PlantGrowth)

# ── Visual check: residuals vs. fitted ───────────────────────────────────────
plot(model_plant, which = 1)
# Points should be randomly scattered around 0 with no fan shape

---

## One-Way ANOVA

**Question:** Does mean plant yield differ across the three treatment groups?  
**H₀:** μ_ctrl = μ_trt1 = μ_trt2  
**H₁:** At least one group mean differs

In [ ]:
# ── Standard one-way ANOVA ────────────────────────────────────────────────────
summary(model_plant)

# ── Welch's one-way ANOVA (if Levene's test was significant) ──────────────────
oneway.test(weight ~ group, data = PlantGrowth, var.equal = FALSE)

# ── Effect size: eta-squared ──────────────────────────────────────────────────
# eta² = SS_between / SS_total
# Small: 0.01 | Medium: 0.06 | Large: 0.14
effectsize::eta_squared(model_plant, partial = FALSE)

# ── Omega-squared (less biased for small samples) ────────────────────────────
effectsize::omega_squared(model_plant)

# ── Interpretation ────────────────────────────────────────────────────────────
# F(df_between, df_within) = X.XX, p = X.XX, eta² = X.XX
# A significant F only tells you *something* differs — use post-hoc tests below

---

## Post-Hoc Tests

A significant ANOVA tells you that group means are not all equal. Post-hoc tests identify *which* pairs differ while controlling for multiple comparisons.

| Test | When to Use |
|---|---|
| Tukey HSD | Equal group sizes; controls familywise error rate; most common default |
| Bonferroni | Conservative; better when testing a small number of planned comparisons |
| Games-Howell | Unequal variances or unequal group sizes (pairs with Welch's ANOVA) |

In [ ]:
# ── Tukey HSD ─────────────────────────────────────────────────────────────────
tukey_result <- TukeyHSD(model_plant)
print(tukey_result)
plot(tukey_result, las = 1)  # confidence intervals for each pairwise comparison
# Intervals not crossing zero indicate significant differences

# ── Bonferroni correction ────────────────────────────────────────────────────
pairwise.t.test(PlantGrowth$weight, PlantGrowth$group,
                p.adjust.method = "bonferroni")

# ── Games-Howell (for unequal variances) ─────────────────────────────────────
rstatix::games_howell_test(PlantGrowth, weight ~ group)

# ── Visualization: mean ± SE with significance brackets ──────────────────────
# Define comparisons to annotate
comparisons <- list(c("ctrl", "trt1"), c("ctrl", "trt2"), c("trt1", "trt2"))

ggboxplot(PlantGrowth, x = "group", y = "weight",
          fill = "group",
          palette = c("#4a8fff", "#4fffb0", "#ffd166"),
          add = "jitter") +
  stat_compare_means(comparisons = comparisons,
                     method = "t.test",
                     p.adjust.method = "bonferroni",
                     label = "p.signif") +
  stat_compare_means(label.y = 7.5) +  # overall ANOVA p-value
  labs(title = "Plant Yield by Treatment",
       subtitle = "One-way ANOVA with Bonferroni post-hoc",
       y = "Dry Weight (g)", x = "Treatment Group") +
  theme(legend.position = "none")

---

## Two-Way ANOVA

**Question:** Does tooth length depend on supplement type, dose, and their interaction?  

Two-way ANOVA tests three hypotheses simultaneously:  
1. Main effect of supplement (OJ vs VC)  
2. Main effect of dose (0.5, 1.0, 2.0 mg)  
3. Interaction effect — does the effect of supplement *depend on* dose?

> **Always check the interaction first.** If the interaction is significant, main effects cannot be interpreted in isolation — the effect of one factor differs across levels of the other.

In [ ]:
# ── Check assumptions for two-way ANOVA ──────────────────────────────────────
model_tooth <- aov(len ~ supp * dose, data = ToothGrowth)
shapiro.test(residuals(model_tooth))
car::leveneTest(len ~ supp * dose, data = ToothGrowth)

# ── Two-way ANOVA with interaction ───────────────────────────────────────────
# Use Type III SS (car::Anova) when group sizes are unequal
car::Anova(model_tooth, type = "III")
# Type I SS (default aov) is order-dependent — avoid for unbalanced designs

# ── Effect sizes ──────────────────────────────────────────────────────────────
effectsize::eta_squared(model_tooth, partial = TRUE)
# partial eta² isolates the variance explained by each factor
# controlling for the other factors in the model

# ── Interaction plot ──────────────────────────────────────────────────────────
ToothGrowth %>%
  group_by(supp, dose) %>%
  summarise(mean_len = mean(len), se = sd(len)/sqrt(n()), .groups = "drop") %>%
  ggplot(aes(x = dose, y = mean_len, color = supp, group = supp)) +
  geom_line(linewidth = 1) +
  geom_point(size = 3) +
  geom_errorbar(aes(ymin = mean_len - se, ymax = mean_len + se),
                width = 0.1) +
  scale_color_manual(values = c("#4a8fff", "#ff6b6b")) +
  labs(title = "Tooth Length by Supplement and Dose",
       subtitle = "Non-parallel lines indicate an interaction effect",
       y = "Mean Tooth Length (mm)",
       x = "Dose (mg)",
       color = "Supplement") +
  theme_minimal()
# Parallel lines: no interaction. Crossing or diverging lines: interaction present.

### Post-Hoc Tests for Two-Way ANOVA

When the interaction is significant, run post-hoc tests **within each level of one factor** rather than across all combinations.

In [ ]:
# ── Simple main effects: effect of dose within each supplement ────────────────
ToothGrowth %>%
  group_by(supp) %>%
  rstatix::anova_test(len ~ dose) %>%
  rstatix::get_anova_table()

# ── Pairwise comparisons within each supplement group ────────────────────────
ToothGrowth %>%
  group_by(supp) %>%
  rstatix::tukey_hsd(len ~ dose)

# ── Faceted boxplot with pairwise annotations ─────────────────────────────────
ggboxplot(ToothGrowth, x = "dose", y = "len",
          fill = "supp",
          palette = c("#4a8fff", "#ff6b6b"),
          facet.by = "supp") +
  stat_compare_means(comparisons = list(c("0.5", "1"), c("1", "2"), c("0.5", "2")),
                     label = "p.signif") +
  labs(title = "Tooth Length by Dose, Faceted by Supplement",
       y = "Tooth Length (mm)", x = "Dose (mg)") +
  theme(legend.position = "none")

---

## Reporting Results

In [ ]:
# ── Extract and format one-way ANOVA results ──────────────────────────────────
anova_table <- summary(model_plant)[[1]]
f_val <- anova_table["group", "F value"]
df1   <- anova_table["group", "Df"]
df2   <- anova_table["Residuals", "Df"]
p_val <- anova_table["group", "Pr(>F)"]
eta2  <- effectsize::eta_squared(model_plant)$Eta2

cat(sprintf(
  "One-way ANOVA: F(%d, %d) = %.2f, p = %.3f, eta² = %.2f\n",
  df1, df2, f_val, p_val, eta2
))

# Standard reporting format:
# F(df_between, df_within) = X.XX, p = .XXX, eta² = .XX
# Follow with post-hoc results: "Treatment 2 produced significantly greater
# yield than control (Tukey HSD, p = .XX, d = .XX)"

---

## Common Pitfalls

**1. Stopping at the omnibus F-test**  
A significant ANOVA only tells you *something* differs. Always follow up with post-hoc tests to identify *which* groups differ and report effect sizes.

**2. Using Type I SS for unbalanced designs**  
R's default `aov()` uses Type I (sequential) sums of squares, which are order-dependent. For unbalanced designs (unequal group sizes), use `car::Anova(model, type = "III")` instead.

**3. Ignoring the interaction in two-way ANOVA**  
If the interaction is significant, interpreting main effects alone is misleading. Always plot the interaction and run simple main effects analysis.

**4. Testing normality on raw data instead of residuals**  
Normality is an assumption about the *residuals* of the model, not the raw observations. Run `shapiro.test(residuals(model))`.

**5. Using standard ANOVA when variances are unequal**  
If Levene's test is significant, use Welch's ANOVA (`oneway.test(var.equal = FALSE)`) and Games-Howell post-hoc tests.

**6. Treating repeated measures as independent**  
If the same subject appears under multiple conditions, a standard ANOVA violates independence. Use repeated measures ANOVA or a linear mixed model (see `03_mixed_effects_models/lmm_basics.ipynb`).

---

## Nonparametric Alternative

If normality of residuals is violated and n is small:

| ANOVA | Nonparametric Equivalent | R Function |
|---|---|---|
| One-way ANOVA | Kruskal-Wallis + Dunn post-hoc | `kruskal.test()` + `dunn.test()` |
| Two-way ANOVA | Scheirer-Ray-Hare test | `rstatix::scheirerRayHare()` |

See `06_nonparametric/rank_tests_multiple_groups.ipynb` for full implementations.

---
*r_methods_library · Samantha McGarrigle · [github.com/samantha-mcgarrigle](https://github.com/samantha-mcgarrigle)*